In [92]:
import pandas as pd

df_pm = pd.read_csv('~/pm_Jan.csv')
df_probs = pd.read_csv('~/probs_Jan.csv')
df_rf_lr = pd.read_csv('~/RF_LR_Jan.csv')
df_rf_lr['RFSpread'] = df_rf_lr['RFHome']  - df_rf_lr['RFAway']
df_rf_lr['LRSpread'] = df_rf_lr['LRHome']  - df_rf_lr['LRAway']


df_actual_results = pd.read_excel('files/algo_picks.xlsx' ,  sheet_name  = 'raw_results')
df_actual_results['HomeTeam']  = df_actual_results['Game'].apply(lambda x: str(x).split(' ')[0])
df_actual_results['AwayTeam']  = df_actual_results['Game'].apply(lambda x: str(x).split(' ')[2])
df_actual_results['Date'] = pd.to_datetime(df_actual_results['Date'])
df_actual_results.dropna(inplace=True)
df_actual_results['ActualSpread'] = df_actual_results['Result'].apply(lambda x: int(str(x).split('-')[0]) - int(str(x).split('-')[1]))
df_actual_results['BeatTheSpread'] = ((df_actual_results['Spread'] +  df_actual_results['ActualSpread'])>0).astype(int)




In [93]:
df = df_pm.merge(df_probs , how='inner' , left_on =['Date','HomeTeam','AwayTeam'], right_on =['Date','HomeTeam','AwayTeam']).\
    merge(df_rf_lr , how='inner' , left_on =['Date','HomeTeam','AwayTeam'], right_on =['Date','HomeTeam','AwayTeam'])


df = df[['Date','HomeTeam','AwayTeam','Probability','OutcomePM','RFSpread','LRSpread']]
df['Date'] = pd.to_datetime(df['Date'])


df = df_actual_results.merge(df , how='inner' , left_on =['Date','HomeTeam','AwayTeam'], right_on =['Date','HomeTeam','AwayTeam'])

df['BeatTheSpread_PM'] = ((df['Spread'] +  df['OutcomePM'])>0).astype(int)
df['BeatTheSpread_RF'] = ((df['Spread'] +  df['RFSpread'])>0).astype(int)
df['BeatTheSpread_LR'] = ((df['Spread'] +  df['LRSpread'])>0).astype(int)

df['ProbabilityCorrect'] = ((df['ActualSpread']>0) &(df['Probability']>0.5) )| ((df['ActualSpread']<0) &(df['Probability']<0.5) )
df['PMCorrect'] = df['BeatTheSpread'] == df['BeatTheSpread_PM']
df['RFCorrect'] = df['BeatTheSpread'] == df['BeatTheSpread_RF']
df['LRCorrect'] = df['BeatTheSpread'] == df['BeatTheSpread_LR']



In [94]:
# probability accuracy score
res = len(df[((df['ActualSpread']>0) &(df['Probability']>0.5) )| ((df['ActualSpread']<0) &(df['Probability']<0.5) )])/ len(df)
print('Probability',res )

# accuracy score - PM
res = len(df[df['BeatTheSpread'] == df['BeatTheSpread_PM']]) / len(df)
print('PM',res )

# accuracy score - RF
res = len(df[df['BeatTheSpread'] == df['BeatTheSpread_RF']]) / len(df)
print('RF',res )

# accuracy score - LR
res = len(df[df['BeatTheSpread'] == df['BeatTheSpread_LR']]) / len(df)
print('LR',res )



Probability 0.7307692307692307
PM 0.5192307692307693
RF 0.5288461538461539
LR 0.4807692307692308


In [91]:
df.groupby('HomeTeam').mean()[['ProbabilityCorrect','PMCorrect','RFCorrect','LRCorrect']]
df.groupby('AwayTeam').mean()[['ProbabilityCorrect','PMCorrect','RFCorrect','LRCorrect']]



,Date,Game,Spread,Result,HomeTeam,AwayTeam,ActualSpread,BeatTheSpread,Probability,OutcomePM,RFSpread,LRSpread,BeatTheSpread_PM,BeatTheSpread_RF,BeatTheSpread_LR,ProbabilityCorrect,PMCorrect,RFCorrect,LRCorrect
0,2020-01-07,CLE vs DET,3.0,113-115,CLE,DET,-2,1,0.23,-3.948236,-0.610848,152.664090,0,1,1,True,False,True,True
1,2020-01-07,BKN vs OKC,-1.0,103-111,BKN,OKC,-8,0,0.34,-2.747103,-1.334198,336.567679,0,0,1,True,True,True,False
2,2020-01-07,MEM vs MIN,-3.5,119-112,MEM,MIN,7,1,0.62,0.159537,0.157480,-757.125842,0,0,0,True,False,False,False
3,2020-01-07,PHX vs SAC,-6.5,103-114,PHX,SAC,-11,0,0.50,1.632138,-0.390062,-179.226863,0,0,0,False,True,True,True
4,2020-01-07,LAL vs NYK,-12.0,117-87,LAL,NYK,30,1,0.77,5.375636,0.592153,286.764712,0,0,1,True,False,False,True
5,2020-01-08,BOS vs SAS,-7.0,114-129,BOS,SAS,-15,0,0.59,-0.230706,1.077489,377.599968,0,0,1,False,True,True,False
6,2020-01-08,ORL vs WAS,-10.0,123-89,ORL,WAS,34,1,0.63,7.784866,-0.505812,101.413427,0,0,1,True,False,False,True
7,2020-01-08,CHA vs TOR,2.5,110-112,CHA,TOR,-2,1,0.50,-1.847314,-0.993788,-166.623847,1,1,0,False,True,True,False
8,2020-01-08,IND vs MIA,1.0,108-122,IND,MIA,-14,0,0.58,-1.699802,0.546609,-443.967090,0,1,0,False,True,False,True
9,2020-01-08,DAL vs DEN,-3.0,106-107,DAL,DEN,-1,0,0.58,4.740714,2.723662,183.749500,1,0,1,False,False,True,False
